![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**1- Make sure that your Python version is 3.3 or higher**

In [19]:
from platform import python_version
print (python_version())

3.8.8


**2- Install SASPy**

In [20]:
pip install saspy

Note: you may need to restart the kernel to use updated packages.


**3- Make sure that the configuration file "sascfg_personal.py" is correctly created**

In [21]:
import saspy, os
print(saspy.__file__.replace('__init__.py', 'sascfg_personal.py'))

/Users/forough/opt/anaconda3/lib/python3.8/site-packages/saspy/sascfg_personal.py


**4- Establish Connection (Need to do this step each time you use SASPy)**

In [28]:
import saspy
sas_session = saspy.SASsession()
sas_session

Using SAS Config named: oda
Error trying to read authinfo file:C:\Users\leGalane\_authinfo
[Errno 2] No such file or directory: 'C:\\Users\\leGalane\\_authinfo'
Did not find key oda in authinfo file:C:\Users\leGalane\_authinfo

Please enter the OMR user id: a01651812@tec.mx
Please enter the password for OMR user : ········
SAS Connection established. Subprocess id is 20956



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = C:\Users\leGalane\anaconda3\lib\site-packages\saspy\sascfg_personal.py
WORK Path             = /saswork/SAS_workAC160000ABC0_odaws02-usw2-2.oda.sas.com/SAS_workA86F0000ABC0_odaws02-usw2-2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 43968


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [29]:
%%SAS sas_session

libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [44]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [45]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
581281,2581282.0,MCLEMORE,BETTY,1.0,38.0,38300.0,University / College,Downtown,6.0,3.0,...,1.0,3.0,40.0,40.0,40.0,0.0,0.0,0.0,1.0,90.0
12738,2012739.0,FIGUEROA,EDWARD,0.0,37.0,185800.0,University / College,Downtown,4.0,0.0,...,1.0,4.0,30.0,30.0,30.0,0.0,0.0,1.0,1.0,10.0


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [31]:
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149457 entries, 3 to 999998
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            149457 non-null  float64
 1   LastName      149454 non-null  object 
 2   FirstName     149457 non-null  object 
 3   Woman         149457 non-null  float64
 4   Age           149457 non-null  float64
 5   Salary        149457 non-null  float64
 6   Education     149457 non-null  object 
 7   City          149457 non-null  object 
 8   SeniorList    149457 non-null  float64
 9   NbActivities  149457 non-null  float64
 10  Referrals     149457 non-null  float64
 11  Recency       68433 non-null   float64
 12  Frequency     68433 non-null   float64
 13  Seniority     68433 non-null   float64
 14  TotalGift     68433 non-null   float64
 15  MinGift       149457 non-null  float64
 16  MaxGift       68433 non-null   float64
 17  GaveLastYear  149457 non-null  float64
 18  AmtL

In [41]:
data_merge[data_merge["TotalGift"].isnull()][["Recency", "Frequency", "Seniority", "TotalGift", "MinGift", "MaxGift"]].describe()

,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift
count,0.0,0.0,0.0,0.0,81024.0,0.0
mean,NaN,NaN,NaN,NaN,0.0,NaN
std,NaN,NaN,NaN,NaN,0.0,NaN
min,NaN,NaN,NaN,NaN,0.0,NaN
25%,NaN,NaN,NaN,NaN,0.0,NaN
50%,NaN,NaN,NaN,NaN,0.0,NaN
75%,NaN,NaN,NaN,NaN,0.0,NaN
max,NaN,NaN,NaN,NaN,0.0,NaN


In [46]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.
#Since we are only going to use the MinGift and the TotalGift from the database, we are going to fill only these NaN values with 0

data_merge[['MinGift', 'TotalGift']] = data_merge[['MinGift', 'TotalGift']].fillna(value=0) 

data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
280756,2280757.0,CARNAHAN,JOHN,0.0,41.0,41300.0,University / College,Downtown,7.0,0.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,1.0,25.0
111145,2111146.0,WILSON,CHARLES,0.0,52.0,138100.0,University / College,City,0.0,0.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,1.0,1.0,20.0


In [47]:
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149457 entries, 3 to 999998
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            149457 non-null  float64
 1   LastName      149454 non-null  object 
 2   FirstName     149457 non-null  object 
 3   Woman         149457 non-null  float64
 4   Age           149457 non-null  float64
 5   Salary        149457 non-null  float64
 6   Education     149457 non-null  object 
 7   City          149457 non-null  object 
 8   SeniorList    149457 non-null  float64
 9   NbActivities  149457 non-null  float64
 10  Referrals     149457 non-null  float64
 11  Recency       68433 non-null   float64
 12  Frequency     68433 non-null   float64
 13  Seniority     68433 non-null   float64
 14  TotalGift     149457 non-null  float64
 15  MinGift       149457 non-null  float64
 16  MaxGift       68433 non-null   float64
 17  GaveLastYear  149457 non-null  float64
 18  AmtL

## Data Partition

In [49]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.5,random_state=5678) # you can change the percentage
train.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
47486,2047487.0,ALIRES,MICHAEL,0.0,66.0,36700.0,High School,Rural,5.0,1.0,...,4.0,4.0,235.0,20.0,150.0,0.0,0.0,0.0,1.0,50.0
301116,2301117.0,TYLER,RENEE,1.0,63.0,190600.0,University / College,Downtown,8.0,3.0,...,2.0,8.0,260.0,10.0,250.0,1.0,30.0,0.0,1.0,10.0



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [51]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','Contact', 'TotalGift', 'MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact', 'TotalGift', 'MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[ 51.07682992  14.04129921 119.85792079 ...  37.78693716  68.2239705
  60.20689539]


In [52]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

63.97154218425214
54373.266622678435
233.18075954649095


## Regression Tree Model（Py）

In [53]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','Contact', 'TotalGift', 'MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact', 'TotalGift', 'MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 51.14299611  27.27249619 139.59760274 ...  52.95749142  47.21358603
  67.88028169]


In [54]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

63.38440342678926
54233.0715638101
232.879950970044


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


In [55]:
#Bayesian Ridge Regression
from sklearn import linear_model

X_train = train[['Age', 'Salary','Contact', 'TotalGift', 'MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact', 'TotalGift', 'MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_valid = validation['AmtThisYear']

BRR_regr = linear_model.BayesianRidge()
BRR_regr.fit(X_train,Y_train)
BRR_regr_predict=BRR_regr.predict(X_valid)

#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

63.97154218425214
54373.266622678435
233.18075954649095


### We are going to use the Bayesian Ridge Regression
---

## Scoring New Data

### Prepare data for scoring

In [56]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [57]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_contact[['TotalGift','MinGift']] = scoring_data_contact[['TotalGift','MinGift']].fillna(value=0) 

#scoring_data_contact.head()

X = scoring_data_contact[['Age', 'Salary','Contact','TotalGift','MinGift', 'AmtLastYear','GaveLastYear','Woman', 'NbActivities' ]] 

regr_predict_contact=BRR_regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,77.299607
1,2000002.0,57.444732
2,2000003.0,67.060042
3,2000004.0,37.791289
4,2000005.0,72.320613


### Predict 'amount given' for members who were not contacted

In [58]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['TotalGift','MinGift']] = scoring_data_nocontact[['TotalGift','MinGift']].fillna(value=0)

#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age', 'Salary','Contact','TotalGift','MinGift','AmtLastYear', 'GaveLastYear','Woman', 'NbActivities' ]] 

regr_predict_nocontact=BRR_regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,81.750054
1,2000002.0,61.895178
2,2000003.0,71.510488
3,2000004.0,42.241736
4,2000005.0,76.771059


In [59]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,77.299607,81.750054
1,2000002.0,57.444732,61.895178
2,2000003.0,67.060042,71.510488


## Exporting Results to a CSV File

In [60]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)